In [28]:
import numpy as np

data = np.load("/home/elena/emcomm/objects_game_test_dataset/[4, 4, 4, 4, 4]_3_distractors.npz")

In [29]:
data

NpzFile '/home/elena/emcomm/objects_game_test_dataset/[4, 4, 4, 4, 4]_3_distractors.npz' with keys: train, train_labels, valid, valid_labels, test...

In [30]:
print(data.files)

['train', 'train_labels', 'valid', 'valid_labels', 'test', 'test_labels', 'n_distractors']


In [31]:
for k in data.files:
    print(k, data[k].shape)


train (2000, 4, 5)
train_labels (2000,)
valid (100, 4, 5)
valid_labels (100,)
test (100, 4, 5)
test_labels (100,)
n_distractors ()


In [32]:
print("First train tuple:\n", data["train"][2])
print("First train label:", data["train_labels"][2])


First train tuple:
 [[4 4 4 3 3]
 [4 4 4 2 3]
 [2 2 2 3 3]
 [1 1 2 4 4]]
First train label: 0


In [33]:
import pprint
pprint.pprint(data["train"][0])

array([[4, 2, 1, 3, 2],
       [4, 2, 4, 1, 3],
       [1, 2, 1, 2, 2],
       [2, 1, 3, 3, 3]])


In [12]:
from datasets import load_from_disk
import numpy as np
from tqdm import tqdm
ds_train = load_from_disk("../../../datasets/coco_train_features_resnet_152")
ds_val = load_from_disk("../../../datasets/coco_val_features_resnet_152")

ds_train, ds_val

(Dataset({
     features: ['coco_url', 'captions', 'image_id', 'features'],
     num_rows: 118287
 }),
 Dataset({
     features: ['coco_url', 'captions', 'image_id', 'features'],
     num_rows: 5000
 }))

In [7]:

# features = np.array(ds["features"])

In [4]:
# def create_tuples(features, n_samples, n_distractors=3):
#     """
#     features: np.array of shape (num_objects, n_features)
#     n_samples: how many tuples to generate
#     n_distractors: number of distractor objects per tuple
#     """
#     num_objects = features.shape[0]
#     tuple_size = n_distractors + 1

#     tuples = []
#     labels = []

#     rng = np.random.default_rng()  # random generator

#     for _ in range(n_samples):
#         # Pick tuple_size unique objects
#         idxs = rng.choice(num_objects, size=tuple_size, replace=False)
#         tuple_vectors = features[idxs]

#         # Pick a random index in the tuple as the target
#         target_idx = rng.integers(0, tuple_size)

#         tuples.append(tuple_vectors)
#         labels.append(target_idx)

#     return np.array(tuples), np.array(labels)


In [5]:
# train_tuples, train_labels = create_tuples(np.array(ds["features"]), n_samples=4096, n_distractors=3)
# valid_tuples, valid_labels = create_tuples(np.array(ds["features"]), n_samples=128, n_distractors=3)
# test_tuples, test_labels   = create_tuples(np.array(ds["features"]), n_samples=128, n_distractors=3)


In [15]:
def create_exhaustive_tuples(features, n_distractors=3, shuffle=True, seed=42):
    """
    Each object is used once as target.
    Distractors are sampled randomly but reproducibly if seed is set.
    """
    num_objects = features.shape[0]
    rng = np.random.default_rng(seed)

    tuples = []
    labels = []

    all_indices = np.arange(num_objects)

    for i in tqdm(range(num_objects)):
        candidates = np.delete(all_indices, i)
        distractor_idxs = rng.choice(candidates, size=n_distractors, replace=False)

        target = features[i]
        distractors = features[distractor_idxs]

        tuple_vectors = np.vstack([target[None, :], distractors])

        if shuffle:
            perm = rng.permutation(n_distractors + 1)
            tuple_vectors = tuple_vectors[perm]
            label = int(np.where(perm == 0)[0][0])
        else:
            label = 0

        tuples.append(tuple_vectors)
        labels.append(label)

    return np.array(tuples), np.array(labels)

In [17]:
train_tuples, train_labels = create_exhaustive_tuples(np.array(ds_train["features"]), n_distractors=3, shuffle=True, seed=42)
test_valid_tuples, test_valid_labels = create_exhaustive_tuples(np.array(ds_val["features"]), n_distractors=3, shuffle=True, seed=42)

valid_tuples = test_valid_tuples[:1500]
valid_labels = test_valid_labels[:1500]
test_tuples = test_valid_tuples[1500:]
test_labels = test_valid_labels[1500:]

100%|██████████| 5000/5000 [00:00<00:00, 18916.39it/s]


In [ ]:
# train_tuples.shape

(4096, 4, 1000)

In [39]:
# for t in train_tuples:
#     for item in t:
#         print(item[0])
#     print("----")

In [ ]:
np.savez_compressed(
    "../../../datasets/coco_npz/coco_full_train_val_152_resnet.npz",
    train=train_tuples,
    train_labels=train_labels,
    valid=valid_tuples,
    valid_labels=valid_labels,
    test=test_tuples,
    test_labels=test_labels,
    n_distractors=3
)